In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, regularizers
import os

### Preparing data

In [3]:
df_train = pd.read_csv('train.csv')
X_train = np.array(df_train)[:, 1:]
assert X_train.shape == (42000, 28*28)
X_train = X_train.reshape((42000, 28, 28, 1))
y_train = np.array(df_train['label'])
y_train_cat = keras.utils.to_categorical(y_train, num_classes=10)

df_test = pd.read_csv('test.csv')
X_test =  np.array(df_test)
assert X_test.shape == (28000, 28*28)
X_test = X_test.reshape((28000, 28, 28, 1))

### Training

In [4]:
model = keras.Sequential([
layers.Conv2D(filters = 32, kernel_size = 5, strides = 1, 
              activation = "relu", input_shape = (28,28,1), kernel_regularizer=regularizers.l2(0.0005)),
layers.Conv2D(filters = 32, kernel_size = 5, strides = 1, use_bias=False),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.MaxPooling2D(pool_size = 2, strides = 2),
layers.Dropout(0.25),
layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, activation = "relu", 
              kernel_regularizer=regularizers.l2(0.0005)),
layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, use_bias=False),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.MaxPooling2D(pool_size = 2, strides = 2),
layers.Dropout(0.25),
layers.Flatten(),
layers.Dense(units = 256, use_bias=False),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.Dense(units = 128, use_bias=False),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.Dense(units = 84, use_bias=False),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.Dropout(0.25),
layers.Dense(units = 10, activation = "softmax")
])
model.compile(optimizer="adam",
              loss=keras.losses.categorical_crossentropy,
              metrics=["accuracy"])

/home/fedimser/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.fit(X_train, y_train_cat, epochs=1, batch_size=1000, validation_split=0.1)      

38/38 ━━━━━━━━━━━━━━━━━━━━ 33s 815ms/step - accuracy: 0.5544 - loss: 1.4351 - val_accuracy: 0.7231 - val_loss: 1.7208


In [6]:
CHECKPOINT_PATH = "model3.weights.h5"
if os.path.exists(CHECKPOINT_PATH):
  model.load_weights(CHECKPOINT_PATH)
  print("Loaded weights.")

Loaded weights.


In [7]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_PATH,
                                                 save_weights_only=True,
                                                 verbose=1)
_ = model.fit(X_train, y_train_cat, epochs=10, batch_size=100, validation_split=0.1, callbacks=[cp_callback])  

Epoch 1/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.9976 - loss: 0.0131
Epoch 1: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 31s 83ms/step - accuracy: 0.9976 - loss: 0.0131 - val_accuracy: 0.9938 - val_loss: 0.0286
Epoch 2/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.9974 - loss: 0.0145
Epoch 2: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 31s 83ms/step - accuracy: 0.9974 - loss: 0.0145 - val_accuracy: 0.9943 - val_loss: 0.0275
Epoch 3/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.9983 - loss: 0.0120
Epoch 3: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 31s 83ms/step - accuracy: 0.9983 - loss: 0.0120 - val_accuracy: 0.9933 - val_loss: 0.0317
Epoch 4/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.9968 - loss: 0.0163
Epoch 4: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 31s 83ms/step - accuracy: 0.9968 - loss: 0.0163 - val_accuracy: 0.9919 - val_loss: 0.0342
Epoc

## Prediction

In [8]:
y_pred = model.predict(X_test)
df_ans = pd.DataFrame({"ImageId":list(range(1,28001)), "Label": np.argmax(y_pred, axis=1)})
df_ans.to_csv('ans_nn.csv',index=False)

875/875 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


Kaggle score is 0.99267.